In [4]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt 
import seaborn as sns
import torch 
from sklearn.utils import shuffle
from causalml.inference.meta import BaseXRegressor, BaseRRegressor, BaseSRegressor, BaseTRegressor
from causalml.inference.nn import CEVAE


ImportError: cannot import name '_ClassNamePrefixFeaturesOutMixin' from 'sklearn.base' (/Users/mawuliagamah/opt/anaconda3/envs/vae/lib/python3.8/site-packages/sklearn/base.py)

In [1]:
# load all lalonde data

nsw = pd.read_stata('"/Users/mawuliagamah/gitprojects/causal_inference/causal_inference/datasets/nsw.dta"')
nswre74_control = pd.read_excel('/Users/mawuliagamah/gitprojects/causal_inference/causal_inference/datasets/nswre74_control.xls')
nswre74_treated = pd.read_excel('/Users/mawuliagamah/gitprojects/causal_inference/causal_inference/datasets/nswre74_treated.xlsx')

nswre74_pd_df = pd.concat([nswre74_control,nswre74_treated]) #Combine treatment into a single control as pandas dataframe

#Shuffle data sets 
nswre74_pd_df = shuffle(nswre74_pd_df).reset_index(drop=True)
nsw_pd_df = shuffle(nsw ).reset_index(drop=True)

NameError: name 'pd' is not defined

In [ ]:
# Define binary features

binfeatures = ['']
